In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import kagglehub
import os

path = kagglehub.dataset_download("chenxaoyu/modelnet-normal-resampled")

print("Path to dataset files:", path)

dataset_root = ""
for root, dirs, files in os.walk(path):
    if 'airplane' in dirs:
        dataset_root = root
        break

print(f"Verified Dataset Root: {dataset_root}")

100%|██████████| 1.65G/1.65G [00:17<00:00, 103MB/s] 

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/chenxaoyu/modelnet-normal-resampled/versions/1
Verified Dataset Root: /root/.cache/kagglehub/datasets/chenxaoyu/modelnet-normal-resampled/versions/1/modelnet40_normal_resampled


In [3]:
!git clone https://github.com/BardiaAkbari/Final-CV-Project.git

Cloning into 'Final-CV-Project'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 43 (delta 7), reused 42 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (43/43), 16.04 MiB | 42.88 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [ ]:
import os
import sys
import torch
import gc

torch.cuda.empty_cache()
gc.collect()

os.chdir('/content/Final-CV-Project/Code')

model_path = 'models/model.py'
if os.path.exists(model_path):
    with open(model_path, 'r') as f:
        code = f.read()
    if "import numpy as np" not in code:
        code = "import numpy as np\n" + code
    if ".view(B*N_new*K, C)" in code:
        code = code.replace(".view(B*N_new*K, C)", ".reshape(B*N_new*K, C)")
    with open(model_path, 'w') as f:
        f.write(code)

models = ['pointtransformer38', 'pointtransformer50']

if 'dataset_root' not in locals():
    possible_root = "/root/.cache/kagglehub/datasets/chenxaoyu/modelnet-normal-resampled/versions/1"
    if os.path.exists(possible_root):
        dataset_root = possible_root
    else:
        dataset_root = os.path.abspath("data/modelnet40_normal_resampled")

for model_name in models:
    print(f"\n{'='*40}")
    print(f"🚀 STARTING TRAINING FOR: {model_name}")
    print(f"{'='*40}\n")

    !mkdir -p data
    !ln -sfn "{dataset_root}" data/modelnet40_normal_resampled

    command = (
        f"python train.py "
        f"--model {model_name} "
        f"--batch_size 8 "
        f"--epoch 150 "
        f"--lr 1e-3 "
        f"--checkpoint_dir ./checkpoints_{model_name} "
    )

    !{command}

    print(f"\n✅ FINISHED TRAINING: {model_name}")

    torch.cuda.empty_cache()

🧹 GPU Memory cleared.

🚀 STARTING TRAINING FOR: pointtransformer38

The size of train data is 9843
The size of test data is 2468
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Epoch 1/150 [Train]:   0% 0/1231 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lo